# Compilation of Notebooks into one
## Original Notebooks and runs in github link: https://github.com/gaylejuntilla/MA3831-A3
NOTE: This notebook has not been run and will probably not run since some codes require specific virtual environments and imports

In [ ]:
# general list of imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import BertTokenizer, BertModel
import pickle
import tensorflow_hub as hub
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import pandas as pd
import numpy as np
import seaborn as sns
import nltk
from nltk.tokenize import sent_tokenize
#nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
#nltk.download('punkt')
#nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
import pickle
import torch
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.optimizers import Adam
from scipy.stats import uniform, randint
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# Scraping

In [ ]:
from selenium import webdriver
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
from datetime import datetime, timezone, timedelta
from selenium.webdriver.chrome.service import Service

class ABCNewsCrawler:
    def __init__(self, start_url, cutoff_date="1 January 2025"):
        """Initialise the instance"""
        self.start_url = start_url
        self.cutoff_date = datetime.strptime(cutoff_date, "%d %B %Y")  # Convert string to datetime
        self.articles = pd.DataFrame(columns=['headline', 'title', 'date', 'content', 'url'])  # Store data
        
        # Initialize the service object for ChromeDriverManager
        service = Service(ChromeDriverManager().install())
        
        # Initialize WebDriver with service and options
        options = webdriver.ChromeOptions()
        options.add_argument('--headless')  # Run headless (no UI)
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        self.driver = webdriver.Chrome(service=service, options=options) 
        
    def run(self):
        """Runs the scraper: loads articles until cutoff date, extracts articles."""
        self.driver.get(self.start_url)
        time.sleep(3)  # Let page load
        
        # Step 1: Load articles until the cutoff date
        self.load_articles_until_cutoff()
        
        # Step 2: Extract articles after loading all
        self.extract_articles()
        
        self.driver.quit()  # Close browser
        #print(self.articles)

    def load_articles_until_cutoff(self):
        """Keep loading more articles until the last article is older than the cutoff date."""
    
        while True:
            # Get all currently loaded articles
            articles = self.driver.find_elements(By.CSS_SELECTOR, 'ul[aria-labelledby="Latest election articles"] li')

            if not articles:
                print("No articles found on page. Stopping.")
                break

            # Find the date of the last article
            try:
                # last article in list
                last_article = articles[-1] 
                # find date attribute
                last_date_str = last_article.find_element(By.XPATH, ".//time[1]").get_attribute("datetime")
                # convert to a datetime object
                last_article_date_utc = datetime.strptime(last_date_str, "%Y-%m-%dT%H:%M:%S.000Z")
                # convert to brisbane time
                last_article_date = self.convert_to_brisbane_time(last_article_date_utc)

                # Stop loading if the last article's date is older than the cutoff
                if last_article_date < self.cutoff_date:
                    print("Reached cutoff date. Stopping 'Load More' clicks.")
                    break

            except Exception as e:
                print(f"Error extracting date from last article: {e}")
                break  # Stop in case of unexpected errors

            # Try clicking "Load More" to get more articles
            try:
                load_more_button = WebDriverWait(self.driver, 5).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[data-component="PaginationLoadMoreButton"]')))
                load_more_button.click()
                time.sleep(2)  # Wait for more articles to load

            except:
                print("No more 'Load More' button. Ending loading.")
                break
    
    def convert_to_brisbane_time(self, utc_datetime):
        """Converts a UTC datetime string to Brisbane time and formats it as 'DD Month YYYY'."""
    
        # Define UTC timezone
        utc_zone = timezone.utc

        # Define Brisbane timezone (UTC+10, no daylight savings)
        brisbane_zone = timezone(timedelta(hours=10))

        # Convert to Brisbane time
        brisbane_datetime = utc_datetime.replace(tzinfo=timezone.utc).astimezone(brisbane_zone)
        
        # Makes the date timezone independent, consistent with the other domains for processing
        brisbane_datetime_naive = brisbane_datetime.replace(tzinfo=None)
        
        # Format as 'DD Month YYYY'
        return brisbane_datetime_naive
    
    def extract_articles(self):
        """Extracts article details: headline, date, content, and URL."""
        
        # Extracts all article elements into a list
        articles = self.driver.find_elements(By.CSS_SELECTOR, 'ul[aria-labelledby="Latest election articles"] li')

        for article in articles:
            try:
                
                # Extract date
                article_date_str = article.find_element(By.XPATH, ".//time[1][@datetime]").get_attribute('datetime')

                # Convert the UTC datetime string to a datetime object
                article_date_utc = datetime.strptime(article_date_str, "%Y-%m-%dT%H:%M:%S.000Z")

                # Convert to UTC+10 and format it to just Day, Month, and Year
                article_date = self.convert_to_brisbane_time(article_date_utc)
                
                # Stop if we've reached the cutoff date
                if article_date < self.cutoff_date:
                    break
                    
                # Extract the article URL
                link_element = article.find_element(By.TAG_NAME, 'a')
                article_url = link_element.get_attribute("href")

                # Extract headline
                headline = article.find_element(By.CSS_SELECTOR, "a[data-component='Link']").text 
                
                # Extract content
                self.driver.execute_script("window.open('');")  # Open new tab
                self.driver.switch_to.window(self.driver.window_handles[1])
                self.driver.get(article_url)
                time.sleep(3) # time to load article
                
                # title is different from the headline so extract title as well
                title = self.driver.find_element(By.CSS_SELECTOR, "h1[data-component='Typography']").text

                content_elements = self.driver.find_elements(By.CSS_SELECTOR, 'div[class*="ArticleRender_article"] p')
                content = " ".join([p.text for p in content_elements])

                # Close tab and switch back
                self.driver.close()
                self.driver.switch_to.window(self.driver.window_handles[0])

                # Add data to DataFrame
                self.articles.loc[len(self.articles)] = [headline, title, article_date, content, article_url]
                print(f"Collected: {headline}")
                
                # wait 10 seconds between each article
                time.sleep(10)

            except Exception as e:
                print(f"Error processing article: {e}")

    
    def save_data_to_file(self):
        """Saves the scraped data to a CSV file."""
        self.articles.to_csv("abc_news_articles.csv", index=False)
        print("Data saved to abc_news_articles.csv.")

In [ ]:
mycrawler = ABCNewsCrawler("https://www.abc.net.au/news/elections/federal-election-2025")
mycrawler.run()
# make sure to save
mycrawler.save_data_to_file()

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
from datetime import datetime
import time

class GuardianScraper:
    def __init__(self, start_url):
        """Initialise the scraper instance"""
        self.start_url = start_url
        self.articles = pd.DataFrame(columns=['headline', 'title', 'content', 'date', 'url'])  # DataFrame to store article data
        # counter to keep track of pagination
        self.current_page = 1

    def scrape_articles(self, url):
        """Scrapes articles from the page and handles pagination."""
        
        while url:
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'lxml') # parses the webpage

            # Step 1: Scrape articles on the current page
            container = soup.find('main')
            articles = container.find_all('li')

            for article in articles:
                try:
                    # Get article URL
                    article_url = self.fetch_article_url(article)
                    print(article_url)

                    # Scrape the article details: title, content, date
                    title, content, date = self.scrape_article_details(article_url)

                    # Add data to DataFrame
                    if title and content and date:
                        new_row = pd.DataFrame({
                                'headline': [title],  # Wrap values in lists
                                'title': [title],
                                'content': [content],
                                'date': [date],
                                'url': [article_url]
                            })
                        self.articles = pd.concat([self.articles, new_row], ignore_index=True)
                        print(f"Collected: {title}")
                    else:
                        print(f"Skipping article: {title}")

                except Exception as e:
                    print(f"Error processing article: {e}")
                    continue

                time.sleep(5)
                
            url = self.get_next_page(soup)
            if not url:
                print("No more pages found, stopping...")
                break
            self.current_page += 1

    
    def fetch_article_url(self, article):
        """method to extract full URL for pagination."""
        base_url = "https://www.theguardian.com"
        article_link = article.find('a')['href']
        if article_link.startswith('/'):
            article_url = base_url + article_link  # Make it a full URL
        else:
            article_url = article_link
        
        return article_url

    def scrape_article_details(self, article_url):
        """Scrapes article content, title, and date from individual article pages."""
        response = requests.get(article_url)
        soup = BeautifulSoup(response.text, 'html.parser')

        try:
            # Extracting title
            title = soup.find('h1').text

            # Extract date text
            date_text = soup.find('span', {'class': 'dcr-u0h1qy'}).text.strip()

            # The date has the time and timezone, so ot needs to be removed
            date_text_without_time_zone = re.sub(r'\s[A-Za-z]+$', '', date_text) 

            # Convert to datetime object
            date_only = ' '.join(date_text_without_time_zone.split()[:4])
            date_obj = datetime.strptime(date_only, "%a %d %b %Y")
            
            # keep formatting consistent
            date = date_obj.strftime("%d %B %Y")

            print(date)  # For example: '30 March 2025'


            # Extracting content
            content_element = soup.find('article')
            paragraphs = content_element.find_all('p')
            content = " ".join([p.text for p in paragraphs]).strip()

            return title, content, date

        except Exception as e:
            print(f"Error processing article: {e}")
            return None, None, None  # Return None in case of an error

    def get_next_page(self, soup):
        """Finds the next page URL from pagination links at the bottom of the page."""
        # Find pagination section
        pagination_links = soup.find('div', {'class': 'dcr-stdtpu'})  

        if pagination_links:
            # Find all numbered page links using regex
            all_pages = pagination_links.find_all('a', string=re.compile(r'\d+')) 

            for page_link in all_pages:
                # if page is greater than the current counter than return URL otherwise None
                if int(page_link.text.strip()) == self.current_page + 1:
                    next_page_url = page_link['href'] # Get the next sequential page
                    # handles the incomplete URL
                    if next_page_url.startswith('/'):
                        next_page_url = "https://www.theguardian.com" + next_page_url
                    return next_page_url

    def save_data_to_file(self, filename='guardian_articles.csv'):
        """Saves the collected articles to a CSV file."""
        self.articles.to_csv(filename, index=False)
        print(f"Data saved to {filename}.")

In [ ]:
start_url = "https://www.theguardian.com/australia-news/australian-election-2025/all"
left_scraper = GuardianScraper(start_url=start_url)
left_scraper.scrape_articles(url=start_url)

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
from datetime import datetime
import time

class SkyNewsScraper:
    def __init__(self, start_url, cutoff_date="1 January 2025"):
        self.start_url = start_url
        self.cutoff_date = datetime.strptime(cutoff_date, "%d %B %Y")  # Convert string to datetime
        self.articles = pd.DataFrame(columns=['headline', 'title', 'content', 'date', 'url'])  # DataFrame to store article data
        self.current_page = 1
        
    def scrape_articles(self, url):
        """Scrapes articles from the page and handles pagination."""
        
        while url:
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'lxml')

            articles = soup.find_all('article', {"class":"storyblock"})

            if not articles:
                print("No articles found on page, stopping...")
                break
            
            for article in articles:
                try:
                    # Extract article info
                    article_info = self.fetch_article_info(article)
                    article_date = datetime.strptime(article_info['date'], "%d %B %Y")  # Convert to datetime object
                    
                    # Check if this article is older than cutoff_date
                    if article_date < self.cutoff_date:
                        print(f"Reached article older than cutoff date: {article_info['date']}, stopping...")
                        return  # Exit function to stop scraping

                    # Scrape article details
                    title, content = self.scrape_article_details(article_info['article_url'])

                    if title and content:
                        new_row = pd.DataFrame({
                            'headline': [article_info['headline']],
                            'title': [title],
                            'content': [content],
                            'date': [article_info['date']],
                            'url': [article_info['article_url']]
                        })
                        self.articles = pd.concat([self.articles, new_row], ignore_index=True)
                        print(f"Collected: {title}")
                    else:
                        print(f"Skipping article: {title}")

                except Exception as e:
                    print(f"Error processing article: {e}")
                    continue

                #time.sleep(5)
                
            # Step 2: Find the next page (pagination)
            url = self.get_next_page(soup)
            if not url:
                print("No more pages found, stopping...")
                break
            self.current_page += 1
    
    def fetch_article_info(self, article):
        """Helper method to extract article information like headline and URL."""
        attributes = article.find('h4', {'class': "storyblock_title"})
        article_url = attributes.find('a')['href']
        headline = attributes.find('a').text
        date_str = article.find('time')['datetime']  # Extract the date string
        date_obj = datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%S%z")  # Convert to datetime object
        date = date_obj.strftime("%d %B %Y")

        return {'headline': headline, 'article_url': article_url, 'date': date} 

    def scrape_article_details(self, article_url):
        """Scrapes article content, title, date, and other details from individual article pages."""
        response = requests.get(article_url)
        soup = BeautifulSoup(response.text, 'html.parser')

        try:
            # Extracting title
            title = soup.find('h1').text

            # Extracting content (Assuming paragraphs are in 'div' with class 'content__article-body')
            content_element = soup.find('div', {'id': 'story-primary'})
            paragraphs = content_element.find_all('p')
            content = " ".join([p.text for p in paragraphs]).strip()

            return title, content

        except Exception as e:
            print(f"Error processing article: {e}")
            return None, None, None  # Return None in case of an error

    def get_next_page(self, soup):
        """Finds the next page URL from pagination links at the bottom of the page."""
        pagination_links = soup.find('ul', {'class': 'page-numbers'})  # Find pagination section

        if pagination_links:
            all_pages = pagination_links.find_all('a', string=re.compile(r'\d+'))  # Find all numbered page links

            # Otherwise, continue to the next page in sequence
            for page_link in all_pages:
                if int(page_link.text.strip()) == self.current_page + 1:
                    return page_link['href']  # Get the next sequential page

    def save_data_to_file(self, filename='skynews_articles.csv'):
        """Saves the collected articles to a CSV file."""
        self.articles.to_csv(filename, index=False)
        print(f"Data saved to {filename}.")
            

In [ ]:
skynews_url = "https://www.skynews.com.au/australia-news/politics"
right_scraper = SkyNewsScraper(start_url=skynews_url)
right_scraper.scrape_articles(url=skynews_url)

# Preprocessing

In [ ]:
guardian = pd.read_csv('guardian_articles.csv')
abc = pd.read_csv('abc_news_articles.csv')
skynews = pd.read_csv('skynews_articles_full.csv')

In [ ]:
def clean_raw_data(df, source):
    """Remove escape sequences and spacing"""
    df = df.copy()
    df.dropna(inplace=True)
    df.drop_duplicates(inplace=True)
    string_columns = ['headline', 'title', 'content']
    for column in string_columns:
         # Remove common escape sequences
        df[column] = df[column].apply(lambda x: re.sub(r'\\[nrt\'"\\]', ' ', str(x)))
        df[column] = df[column].apply(lambda x: re.sub(r'\s+', ' ', str(x)).strip())
        # Specifically replace escaped apostrophes (\' becomes ')
        df[column] = df[column].apply(lambda x: re.sub(r'\\"', '"', str(x)))

    df['date'] = df['date'].apply(parse_date)
    df['date'] = df['date'].dt.date
    df.reset_index(drop=True, inplace=True)
    df['source'] = source
    return df

In [ ]:
def parse_date(date_str):
    """Convert the dates to datetime objects"""
    try:
        # Try parsing as date-time format (like '2025-04-01 18:18:43')
        return pd.to_datetime(date_str, errors='raise')  # Raise error if parsing fails
    except Exception:
        try:
            # If the first attempt fails, try date-only format (like '10 March 2025')
            return pd.to_datetime(date_str, format='%d %B %Y', errors='coerce')
        except Exception:
            # If both attempts fail, return NaT (Not a Time)
            return pd.NaT
guardian_df = clean_raw_data(guardian, 'theguardian')
abc_df = clean_raw_data(abc, 'abc')
skynews_df = clean_raw_data(skynews, 'skynews')

In [ ]:
# selecting every 5th row to reduce size
skynews_df = skynews_df.iloc[::5]

# NER

In [ ]:
nlp = spacy.load("en_core_web_sm")
corpus = pd.read_csv('corpus.csv')
corpus_df = corpus.copy()

In [ ]:
party_leader_pairs = {
    "Australian Labor Party": "Anthony Albanese",
    "Liberal Party": "Peter Dutton",
    "National Party": "David Littleproud",
    "Coalition": "Peter Dutton",
    "Australian Greens": "Adam Bandt",
    "One Nation": "Pauline Hanson",
    "Australia's Voice": "Fatima Payman",
    "Centre Alliance": "Rebekha Sharkie",
    "David Pocock": "David Pocock",
    "Jacqui Lambie Network": "Jacqui Lambie",
    "Katter's Australian Party": "Robbie Katter",
    "People First Party": "Gerard Rennick",
    "United Australia Party": "Ralph Babet",
    "Animal Justice Party": "Angela Pollard",
    "Australian Christians": "Maryka Groenewald",
    "Australian Citizens Party": "Craig Isherwood",
    "Australian Democrats": "Lyn Allison",
    "Better Together Party": "Lucy Bradlow",
    "Fusion Party": "Drew Wolfendale",
    "Family First Party": "Lyle Shelton",
    "The Great Australian Party": "Rod Culleton",
    "Indigenous-Aboriginal Party": "'Uncle' Owen Whyman",
    "Kim for Canberra": "Kim Rubenstein",
    "Legalise Cannabis": "Michael Balderstone",
    "Libertarian Party": "Anthony Bull",
    "Shooters, Fishers, and Farmers Party": "Robert Brown",
    "Socialist Alliance": "Jacob Andrewartha",
    "Sustainable Australia Party": "Celeste Ackerly",
    "Trumpet of Patriots": "Suellen Wrightson",
    "Victorian Socialists": "Collective leadership",
    "Katter's Australian Party": "Bob Katter",
    "Independents": "Independents"
}
abbreviations_parties = {
    "Greens": "Australian Greens",
    "Labor": "Australian Labor Party",
    "Liberal": "Coalition",
    "National": "Coalition"
}
abbreviations_leaders = {
    "Dutton": "Peter Dutton",
    "Albanese": "Anthony Albanese",
    "Bandt": "Adam Bandt"
}

In [ ]:
def add_party_and_leader_entities(nlp):
    """Creates a rule-based system for important entities"""
    # Creating a new named entity ruler before the NER pipe
    ruler = nlp.add_pipe("entity_ruler", before="ner")
    
    # Prepare a list of patterns for full names and abbreviations
    patterns = []
    
    # Add patterns for full party names and their leaders
    for party, leader in party_leader_pairs.items():
        patterns.append({"label": "ORG", "pattern": party})  # Party as ORG
        patterns.append({"label": "PERSON", "pattern": leader})  # Leader as PERSON
    
    # Add patterns for abbreviations
    for short_party_name, full_party_name in abbreviations_parties.items():
        patterns.append({"label": "ORG", "pattern": short_party_name})
    
    for short_name, full_name in abbreviations_leaders.items():
        patterns.append({"label": "PERSON", "pattern": short_name}) 
    
    # Add the patterns to the entity ruler
    ruler.add_patterns(patterns)

In [ ]:
def ner_with_spacy(text):
    nlp = spacy.load("en_core_web_sm")
    add_party_and_leader_entities(nlp)
    people = []
    parties = []
    doc = nlp(text)
    
    for ent in doc.ents:
        if ent.label_ == "PERSON":
            people.append(ent.text)
        elif ent.label_ == "ORG":
            if ent.text in party_leader_pairs.keys() or ent.text in abbreviations_parties.keys():
                parties.append(ent.text)
    return people, parties

In [ ]:
corpus_df[['headline_people', 'headline_parties']] = corpus_df['headline'].apply(lambda x: pd.Series(ner_with_spacy(x)))
corpus_df[['content_people', 'content_parties']] = corpus_df['content'].apply(lambda x: pd.Series(ner_with_spacy(x)))

In [ ]:
corpus_df['headline_people'] = corpus_df['headline_people'].apply(lambda x: list(set(x)))
corpus_df['headline_parties'] = corpus_df['headline_parties'].apply(lambda x: list(set(x)))
corpus_df['content_people'] = corpus_df['content_people'].apply(lambda x: list(set(x)))
corpus_df['content_parties'] = corpus_df['content_parties'].apply(lambda x: list(set(x)))

In [ ]:
corpus_df.to_csv('corpus_NER.csv', index=False)

# VADER

In [ ]:
analyzer = SentimentIntensityAnalyzer()

# Function to apply VADER sentiment analysis to text
def get_vader_sentiment(text):
    sentiment = analyzer.polarity_scores(text) # Get sentiment scores
    return sentiment['compound']  # Compound score is the overall sentiment

In [ ]:
corpus_df['headline_score'] = corpus_df['headline'].apply(get_vader_sentiment)
corpus_df['headline_sentiment'] = corpus_df['headline_score'].apply(lambda x: 'positive' if x>0.05 else ('negative' if x< -0.05 else 'neutral'))

# ELMo Embeddings
## Article content

In [ ]:
elmo = hub.load("https://tfhub.dev/google/elmo/2")

In [ ]:
def truncate_content(content, token_limit=800):
    # Initialize variables to count tokens and store truncated sentences
    total_tokens = 0
    truncated_sentences = []
    sentences = sent_tokenize(content)
    
    for sentence in sentences:
        sentence_tokens = len(sentence.split())  # Token count of the sentence
        
        # Check if adding this sentence will exceed the token limit
        if total_tokens + sentence_tokens > token_limit:
            break  # Stop if we've reached the limit
        truncated_sentences.append(sentence)
        total_tokens += sentence_tokens
    
    # Join the truncated sentences back into a string
    return ' '.join(truncated_sentences)
corpus_df['content'] = corpus_df['content'].apply(truncate_content)

In [ ]:
def get_elmo_embeddings_batch(text_batch):
    """Function to get ELMo embeddings for a batch of text"""
    elmo_input = tf.convert_to_tensor(text_batch, dtype=tf.string)
    
    # Get the embeddings from ELMo
    elmo_output = elmo.signatures['default'](text=elmo_input)
    embeddings = elmo_output['elmo']
    
    return embeddings

def process_in_batches(df, batch_size=16, max_length=800):
    """
    Process the content column in batches, getting ELMo embeddings for each batch.
    Pads the embeddings to ensure consistent shapes for concatenation.
    """
    embeddings = []
    total_rows = len(df)
    
    for start in range(0, total_rows, batch_size):
        # Create batch slice
        end = min(start + batch_size, total_rows)
        batch_text = df['content'][start:end].tolist()
        
        # Get ELMo embeddings for the batch
        try:
            batch_embeddings = get_elmo_embeddings_batch(batch_text)
            
            # Pad to max_length if sentence is shorter, truncate if longer
            batch_embeddings_padded = pad_sequences(batch_embeddings.numpy(), 
                                                     maxlen=max_length, 
                                                     dtype='float32', 
                                                     padding='post', 
                                                     truncating='post')
            
            embeddings.append(batch_embeddings_padded)  # Add the padded embeddings
        except Exception as e:
            print(f"Error processing batch {start}-{end}: {e}")
            embeddings.append(None)  # Append None in case of failure
        
        # Print progress every 10 batches
        if (start // batch_size) % 10 == 0:
            print(f"Processed {start} to {end} rows...")
    
    # Concatenate all batches together
    embeddings = np.concatenate(embeddings, axis=0)
    
    return embeddings

# Process just the 'content' column and return embeddings as a separate variable
elmo_embeddings = process_in_batches(corpus_df, batch_size=16, max_length=800)

# Now, elmo_embeddings holds the embeddings for the 'content' column only
print(elmo_embeddings.shape)

## Headline embeddings

In [ ]:
# Headline embeddings for bias classifier
def elmo_tokenize(texts):
    """Use ELMo's internal tokenization (same as used in the model)."""
    embeddings = elmo.signatures['default'](tf.convert_to_tensor(texts))
    return embeddings['elmo']

def get_max_tokenized_length(df):
    """Calculate the max tokenized length using ELMo's internal tokenization."""
    max_length = 0
    for text in df['headline']:
        # Tokenize text using ELMo's tokenizer (embedding)
        tokens = elmo_tokenize([text])  # Tokenize sentence
        token_length = tokens.shape[1]  # Get token count (second dimension)
        max_length = max(max_length, token_length)
    return max_length

def process_headline_embeddings(df, batch_size=32):
    """Process the headline column in batches, getting ELMo embeddings."""
    embeddings = []
    total_rows = len(df)
    
    # Calculate the max tokenized length once
    max_tokenized_len = get_max_tokenized_length(df)
    print(f"Max tokenized sentence length: {max_tokenized_len}")
    
    for start in range(0, total_rows, batch_size):
        # Create batch slice
        end = min(start + batch_size, total_rows)
        batch_text = df['headline'][start:end].tolist()
        
        # Get ELMo embeddings for the batch
        try:
            batch_embeddings = elmo_tokenize(batch_text)
            
            batch_embeddings_padded = np.array([np.pad(
                emb.numpy(), 
                ((0, max_tokenized_len - emb.shape[0]), (0, 0)),  # Pad to max length
                'constant') for emb in batch_embeddings])
            
            embeddings.append(batch_embeddings_padded)  # Add the padded embeddings
        except Exception as e:
            print(f"Error processing batch {start}-{end}: {e}")
            embeddings.append(None)  # Append None in case of failure
        
        # Print progress every 10 batches
        if (start // batch_size) % 10 == 0:
            print(f"Processed {start} to {end} rows...")
    
    # Concatenate all batches together
    embeddings = np.concatenate(embeddings, axis=0)
    
    return embeddings

# Call this function to process the embeddings
elmo_headline_embeddings = process_headline_embeddings(corpus_df, batch_size=32)

# Check the shape (should be num_samples, max_tokenized_len, 1024)
print(elmo_headline_embeddings.shape)

# Sentiment Classifier

In [ ]:
y_sentiment = pd.get_dummies(corpus_df['headline_sentiment']).to_numpy()
corpus2 = corpus_df.drop(manual_indices)
y_sentiment = pd.get_dummies(corpus2['headline_sentiment']).to_numpy()
y2 = np.argmax(y_sentiment, axis=1)

In [ ]:
def train_lstm_classifier(X, y, max_length, embedding_dim, num_classes=3, 
                          lstm_units=128, dropout_rate=0.3, dense_units=4,
                          batch_size=16, epochs=5, validation_split=0.2):
    """
    Train an LSTM model on the given data.
    
    Parameters:
        X (ndarray): Input embeddings
        y (ndarray): One-hot encoded labels
        max_length (int): Max sequence length
        embedding_dim (int): Embedding dimensionality (e.g. 1024 for ELMo, 768 for BERT)
        num_classes (int): Number of output classes
        lstm_units (int): Number of LSTM units
        dropout_rate (float): Dropout rate
        dense_units (int): Units in dense layer
        batch_size (int): Batch size
        epochs (int): Number of training epochs
        validation_split (float): Fraction of data used for validation
    """
    indices = np.arange(len(X))
    X_train, X_test, y_train, y_test, idx_train, idx_test = train_test_split(
        X, y, indices, test_size=validation_split, random_state=42
    )

    model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(input_shape=(max_length, embedding_dim)),
        tf.keras.layers.LSTM(lstm_units, return_sequences=False),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Dense(dense_units, activation='relu'),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(loss='categorical_crossentropy',
                  optimizer=Adam(learning_rate=0.001),
                  metrics=['accuracy'])

    history = model.fit(X_train, y_train,
                        epochs=epochs,
                        batch_size=batch_size,
                        validation_data=(X_test, y_test))
    
    y_pred_probs = model.predict(X_test)
    # Converts predictions into single classes from 0-2
    y_pred = np.argmax(y_pred_probs, axis=1)
    y_true = np.argmax(y_test, axis=1)

    evaluation = model.evaluate(X_test, y_test)
    print(f"Test Loss: {evaluation[0]}, Test Accuracy: {evaluation[1]}")

    return model, history, evaluation, y_pred, y_true

In [ ]:
model_elmo_sent, history_elmo_sent, eval_elmo_sent, y_pred_elmo_sent, y_true_elmo_sent = train_lstm_classifier(
    X=elmo_embeddings,
    y=y_sentiment,
    max_length=800,
    embedding_dim=1024
)
print(classification_report(y_true_elmo_sent, y_pred_elmo_sent, zero_division=1))

In [ ]:
def plot_training_history(history):
    plt.figure(figsize=(12, 5))

    # Accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Acc')
    plt.plot(history.history['val_accuracy'], label='Val Acc')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Training vs Validation Accuracy')
    plt.legend()

    # Loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Val Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training vs Validation Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()

plot_training_history(history_elmo_sent)

In [ ]:
def vader_to_label(score):
    """Mapping the headline VADER scores to match the supervised model scores"""
    if score <= -0.05:
        return 0  # negative
    elif score >= 0.05:
        return 2  # positive
    else:
        return 1  # neutral

corpus2['vader_to_lstm'] = corpus_df['headline_score'].apply(vader_to_label)

In [ ]:
# returns difference between 0-2
corpus2['sentiment_diff'] = abs(corpus2['vader_to_lstm'] - corpus2['content_sentiment'])

In [ ]:
corpus2['sentiment_mismatch'] = corpus2['vader_to_lstm'] != corpus2['content_sentiment']

## Visualising sentiment and source and NER

In [ ]:
label_map = {0: "same", 1: "more postive/negative", 2: "opposite"}

ax = sns.countplot(x='sentiment_diff', data=corpus2, palette='muted')
plt.title('Difference Between Headline and Content Sentiment')
ax.set_xticklabels([label_map.get(int(label.get_text()), "") for label in ax.get_xticklabels()])
plt.xlabel('Sentiment Difference (Headline - Content)')
plt.ylabel('Number of Articles')
plt.show()

In [ ]:
import ast
cols_to_fix = ['headline_people', 'headline_parties', 'content_people', 'content_parties']
for col in cols_to_fix:
    corpus2[col] = corpus2[col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

In [ ]:
leader_aliases = {
    "Albanese": "Anthony Albanese",
    "Anthony Albanese": "Anthony Albanese",
    "Dutton": "Peter Dutton",
    "Peter Dutton": "Peter Dutton",
    "Bandt": "Adam Bandt",
    "Adam Bandt": "Adam Bandt",
    "Morrison": "Scott Morrison",
    "Scott Morrison": "Scott Morrison",
    "Marles": "Richard Marles",
    "Richard Marles": "Richard Marles",
    "Wong": "Penny Wong",
    "Penny Wong": "Penny Wong"
}

def normalize_leaders(person_list):
    return [leader_aliases[name] for name in person_list if name in leader_aliases]

corpus2['headline_people'] = corpus2['headline_people'].apply(normalize_leaders)
filtered_df = corpus2[corpus2['headline_people'].apply(lambda x: len(x) > 0)]
filtered_df.to_csv('filtered_df.csv', index=False)

In [ ]:
import matplotlib.cm as cm 
import matplotlib.pyplot as plt
puor_colors = cm.get_cmap('PuOr', 3)
colors = [puor_colors(i) for i in range(3)]  # RGBA tuples
colors_hex = [plt.matplotlib.colors.to_hex(c) for c in colors]

exploded_df = filtered_df.explode('headline_people')
sentiment_counts = exploded_df.groupby(['headline_people', 'headline_sentiment']).size().unstack(fill_value=0)
sentiment_counts.plot(kind = 'bar', colormap = 'PuOr', width = 1)
plt.xlabel('NER political leaders')
plt.ylabel('VADER Sentiment Count')
plt.legend()

plt.show()

In [ ]:
source_counts = exploded_df.groupby(['source', 'headline_sentiment']).size().unstack(fill_value=0)
source_counts.plot(kind = 'bar', colormap = 'PuOr', width = 1)
plt.legend()
plt.ylabel('VADER Sentiment Count')
plt.xlabel('News Outlet')
plt.show()

In [ ]:
source_and_leader_counts = exploded_df.groupby(['source', 'headline_sentiment', 'headline_people']).size().unstack(fill_value=0)

In [ ]:
source_and_leader_counts[['Anthony Albanese', 'Peter Dutton']]

In [ ]:
sentiment_order = ['negative', 'neutral', 'positive']
corpus_df['headline_sentiment'] = pd.Categorical(
    corpus_df['headline_sentiment'],
    categories=sentiment_order,
    ordered=True
)

In [ ]:
sns.countplot(
    data=corpus_df,
    x='headline_sentiment',
    order=sentiment_order,
    palette=colors
)
plt.xlabel('Headline Sentiment')
plt.ylabel('Count')
plt.show()

In [ ]:
sns.countplot(
    data=corpus_df,
    x='source',
    order=['theguardian', 'abc', 'skynews'],
    palette=colors
)
plt.xlabel('News Outlet')
plt.ylabel('Count')
plt.title('Distribution of Corpus Articles by News Outlet')
plt.show()

In [ ]:
sns.countplot(
    data=corpus2.iloc[idx_train],
    x='source',
    order=['theguardian', 'abc', 'skynews'],
    palette=colors
)
plt.xlabel('News Outlet')
plt.ylabel('Count')
plt.title('Distribution of Sample Articles by News Outlet')
plt.show()
#sns.histplot(corpus2.iloc[idx_train]['source'], color='#b76b3e')

# Bias Classifier
## Baseline Model

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

# Download NLTK resources
#nltk.download('punkt')
#nltk.download('stopwords')
#nltk.download('wordnet')

# Initialize lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def nltk_tokenizer(text):
    """tokenizer and preprocessing using NLTK"""
    # Tokenize the text
    tokens = nltk.word_tokenize(text)
    
    # Remove stopwords and non-alphabetic words, then lemmatize
    cleaned_tokens = [
        lemmatizer.lemmatize(token.lower()) 
        for token in tokens 
        if token.isalpha() and token.lower() not in stop_words
    ]
    
    return cleaned_tokens

# Vectorize the headlines using TF-IDF
vectorizer = TfidfVectorizer(tokenizer=nltk_tokenizer, lowercase=False)
X_tfidf = vectorizer.fit_transform(corpus_df['headline'])

# Encode the target labels using onehot encoding
y_encoded = pd.get_dummies(corpus_df['source']).to_numpy()  # Encode the categorical labels
y_encoded_flat = np.argmax(y_encoded, axis=1)
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y_encoded_flat, test_size=0.2, random_state=42)

# Train a Logistic Regression model
model_lr = LogisticRegression(max_iter=1000, multi_class='ovr')  # 'ovr' for multi-class
model_lr.fit(X_train, y_train)

# Make predictions
y_pred = model_lr.predict(X_test)

# Evaluate the model using classification metrics
print(classification_report(y_test, y_pred))

## BERT Embeddings

In [ ]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')
headlines = corpus_df['headline'].tolist()

max_len = max(len(bert_tokenizer.tokenize(h)) for h in headlines)


def get_bert_embeddings(text_list, max_len=max_len):
    """
    Tokenizes and returns BERT embeddings for a list of texts with optional max_len.
    """

    # Tokenize with dynamic max length padding
    inputs = bert_tokenizer(
        text_list,
        padding='max_length',
        truncation=True,
        max_length=max_len,
        return_tensors='pt' # pytorch tensors
    )

    with torch.no_grad(): # does not compute gradients
        outputs = bert_model(**inputs)
    
    # Shape: (batch_size, max_len, 768)
    return outputs.last_hidden_state

def get_bert_embeddings_numpy(text_list, max_len=None):
    """
    Same as get_bert_embeddings but returns NumPy array.
    """
    embeddings = get_bert_embeddings(text_list, max_len)
    return embeddings.detach().cpu().numpy()

bert_headline_embeddings = get_bert_embeddings(headlines)
bert_headline_embeddings.shape

## Hyperparameter Tuning

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

def create_model(units=128, learning_rate=0.001, drop_rate=0.2, dense_units=64):
    """LSTM model for tuning"""
    model = Sequential()
    model.add(LSTM(units=units, input_shape=(32, 768)))  # change for BERT or ELMo input
    model.add(Dropout(rate=drop_rate))
    model.add(Dense(dense_units, activation='relu'))
    model.add(Dense(3, activation='softmax'))  # 3 classes in output layer
    model.compile(optimizer=Adam(learning_rate=learning_rate), 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])
    return model

def grid_search_model(X, y, param_grid, verbose=0):
    """Function for hyperparameter tuning"""
    # Fix random seed for reproducibility
    seed = 7
    tf.random.set_seed(seed)

    # Create the KerasClassifier
    model = KerasClassifier(model=create_model, epochs=10, batch_size=16, verbose=0)

    # Create GridSearchCV object
    grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)

    # Fit the grid search with early stop depending on loss
    early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    grid_result = grid.fit(X, y, callbacks=[early_stop], validation_split=0.2)

    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

    # Show the mean and standard deviation of each parameter setting
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']

    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))

    return grid_result.best_params_, grid_result.best_score_

# Define the grid search parameters
learn_rate = [0.001, 0.01, 0.1]
unit_size = [64, 128]  # Different sizes for LSTM units
drop_rate = [0.2, 0.3, 0.5]
dense_units = [16, 32, 64]

# Param grid for grid search
param_grid = dict(optimizer__learning_rate=learn_rate, 
                  model__units=unit_size,
                  model__drop_rate=drop_rate, 
                  model__dense_units=dense_units)

In [ ]:
best_params_bert, best_score_bert = grid_search_model(bert_headline_embeddings, y_source, param_grid)

In [ ]:
best_params_elmo, best_score_elmo = grid_search_model(bert_headline_embeddings, y_source, param_grid)

## Fitting Optimised Models
### BERT

In [ ]:
y_source = pd.get_dummies(corpus_df['source']).to_numpy()
def train_lstm_classifier(X, y, max_length, embedding_dim, num_classes=3, 
                          lstm_units=64, dropout_rate=0.3, dense_units=32,
                          batch_size=16, epochs=6, validation_split=0.2):
    """
    Train an LSTM model on the given data.
    
    Parameters:
        X (ndarray): Input embeddings of shape (samples, max_length, embedding_dim)
        y (ndarray): One-hot encoded labels
        max_length (int): Max sequence length
        embedding_dim (int): Embedding dimensionality (e.g. 1024 for ELMo, 768 for BERT)
        num_classes (int): Number of output classes
        lstm_units (int): Number of LSTM units
        dropout_rate (float): Dropout rate
        dense_units (int): Units in dense layer
        batch_size (int): Batch size
        epochs (int): Number of training epochs
        validation_split (float): Fraction of data used for validation
    
    Returns:
        model (tf.keras.Model): Trained model
        history: Training history
        evaluation: (loss, accuracy) on test set
    """
    indices = np.arange(len(X))
    X_train, X_test, y_train, y_test, idx_train, idx_test = train_test_split(
        X, y, indices, test_size=validation_split, random_state=42
    )

    model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(input_shape=(max_length, embedding_dim)),
        tf.keras.layers.LSTM(lstm_units, return_sequences=False),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Dense(dense_units, activation='relu'),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(loss='categorical_crossentropy',
                  optimizer=Adam(learning_rate=0.001),
                  metrics=['accuracy'])

    history = model.fit(X_train, y_train,
                        epochs=epochs,
                        batch_size=batch_size,
                        validation_data=(X_test, y_test))
    
    y_pred_probs = model.predict(X_test)
    y_pred = np.argmax(y_pred_probs, axis=1)
    y_true = np.argmax(y_test, axis=1)

    evaluation = model.evaluate(X_test, y_test)
    print(f"Test Loss: {evaluation[0]}, Test Accuracy: {evaluation[1]}")

    return model, history, evaluation, y_pred, y_true

In [ ]:
model_bert, history_bert, eval_bert, y_pred_bert, y_true_bert = train_lstm_classifier(
    X=bert_headline_embeddings,
    y=y_source,
    max_length=max_len,  # dynamically determined
    embedding_dim=768
)
print("BERT Report")
print(classification_report(y_true_bert, y_pred_bert))

In [ ]:
def plot_training_history(history):
    plt.figure(figsize=(12, 5))

    # Accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Acc')
    plt.plot(history.history['val_accuracy'], label='Val Acc')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Training vs Validation Accuracy')
    plt.legend()

    # Loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Val Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training vs Validation Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()

plot_training_history(history_bert)

### ELMo

In [ ]:
# Best: 0.711185 using {'model__dense_units': 32, 'model__drop_rate': 0.2, 'model__units': 64, 'optimizer__learning_rate': 0.01}
model_elmo, history_elmo, eval_elmo, y_pred_elmo, y_true_elmo = train_lstm_classifier(
    X=elmo_headline_embeddings,
    y=y_source,
    max_length=29,
    embedding_dim=1024
)
print("ELMo Report")
print(classification_report(y_true_elmo, y_pred_elmo))
plot_training_history(history_elmo)

## Extra Optional XGBoost Model (ran out of words to include this)

In [ ]:
#!pip install xgboost
import xgboost as xgb

X_train_xgb, X_test_xgb, y_train_xgb, y_test_xgb = train_test_split(X_cls, y_source, test_size=0.2, random_state=42)

# Step 4: Train an XGBoost model on the embeddings
xgb_model = xgb.XGBClassifier()
xgb_model.fit(X_train_xgb, y_train_xgb)

# Step 5: Evaluate the model
y_pred_xgb = xgb_model.predict(X_test_xgb)
print(classification_report(y_test_xgb, y_pred_xgb))